# ProteinMCP — Binder Design Workflow

Design protein binders using BindCraft with GPU-accelerated deep learning for de novo binder generation.

| Component | Description |
|-----------|-------------|
| **BindCraft** | De novo binder design using RFdiffusion + ProteinMPNN + AlphaFold2 |
| **Async Jobs** | GPU-accelerated design with asynchronous job submission and monitoring |
| **Quality Metrics** | pLDDT, pAE, interface scores, composite quality assessment |

**Prerequisites:** Docker (with GPU support), Claude Code CLI, ProteinMCP installed locally.

**Links:** [GitHub](https://github.com/charlesxu90/ProteinMCP) · [BindCraft](https://github.com/martinpacesa/BindCraft)

---

## Job configs

In [1]:
# ── User Configuration ──
TARGET_NAME = "PDL1"
TARGET_CHAINS = "A"
BINDER_LENGTH = 130
NUM_DESIGNS = 3

# Optional: set API key here or in .env. If unset, Claude CLI uses your logged-in account.
ANTHROPIC_API_KEY = ""
CLAUDE_MODEL = "claude-sonnet-4-6"

## Import utility and define functions

In [2]:
import os
import subprocess
import json
import select

# ---------- Streaming command runner ----------
def run_cmd(cmd, cwd=None):
    """Run a shell command and stream stdout/stderr line-by-line in real time."""
    proc = subprocess.Popen(
        cmd, shell=True, cwd=cwd,
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        bufsize=1, text=True,
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    proc.wait()
    if proc.returncode != 0:
        print(f"\n\u26a0\ufe0f  Command exited with code {proc.returncode}")
    return proc.returncode

# ---------- Claude streaming helper ----------
def _display_claude_line(line):
    """Parse a single stream-json line from Claude CLI and print progress."""
    if not line.strip():
        return
    try:
        data = json.loads(line)
        msg_type = data.get('type', '')
        subtype = data.get('subtype', '')

        if msg_type == 'system':
            if subtype == 'init':
                session_id = data.get('session_id', '')[:8]
                print(f"  \U0001f916 Session started: {session_id}...", flush=True)
            elif subtype != 'transcript':
                print(f"  \u2699\ufe0f  System: {subtype}", flush=True)

        elif msg_type == 'assistant':
            message = data.get('message', {})
            for block in message.get('content', []):
                block_type = block.get('type', '')
                if block_type == 'thinking':
                    text = block.get('thinking', '')[:100]
                    print(f"  \U0001f4ad Thinking: {text}...", flush=True)
                elif block_type == 'text':
                    lines = block.get('text', '').strip().split('\n')
                    for tl in lines[:5]:
                        if tl.strip():
                            print(f"  {tl}", flush=True)
                    if len(lines) > 5:
                        print(f"  ... ({len(lines) - 5} more lines)", flush=True)
                elif block_type == 'tool_use':
                    tool_name = block.get('name', 'unknown')
                    tool_input = block.get('input', {})
                    if tool_name == 'Bash':
                        print(f"  \U0001f527 Bash: {tool_input.get('command', '')[:80]}", flush=True)
                    elif tool_name in ('Read', 'Write', 'Edit'):
                        print(f"  \U0001f4d6 {tool_name}: {tool_input.get('file_path', '')}", flush=True)
                    elif tool_name.startswith('mcp__'):
                        print(f"  \U0001f50c MCP: {tool_name}", flush=True)
                    else:
                        print(f"  \U0001f527 {tool_name}", flush=True)

        elif msg_type == 'user':
            for block in data.get('message', {}).get('content', []):
                if block.get('type') == 'tool_result':
                    if block.get('is_error', False):
                        err = block.get('content', '')
                        err = err[:100] if isinstance(err, str) else str(err)[:100]
                        print(f"  \u274c Error: {err}", flush=True)
                    else:
                        content = block.get('content', '')
                        if isinstance(content, str) and content.strip():
                            first = content.strip().split('\n')[0][:80]
                            if first:
                                print(f"  \u2705 Result: {first}", flush=True)
                        else:
                            print(f"  \u2705 Done", flush=True)

        elif msg_type == 'result':
            if subtype == 'success':
                print(f"  \u2705 Completed successfully", flush=True)
            elif subtype == 'error':
                print(f"  \u274c Error: {data.get('error', 'Unknown')}", flush=True)

    except json.JSONDecodeError:
        if line.strip():
            print(f"  {line}", flush=True)


def run_claude(prompt, allowed_tools=None, cwd=None):
    """Run Claude CLI with real-time streaming output.

    Args:
        prompt: The prompt text to send to Claude (passed via stdin).
        allowed_tools: Comma-separated tool names, e.g. "Bash,Read,Write".
        cwd: Working directory for the claude process.

    Returns:
        Process return code (0 = success).
    """
    cmd = [
        "claude",
        "--model", CLAUDE_MODEL,
        "-p", "-",
        "--output-format", "stream-json",
        "--verbose",
        "--dangerously-skip-permissions",
    ]
    if allowed_tools:
        cmd += ["--allowedTools", allowed_tools]

    print(f"  \U0001f916 Claude model: {CLAUDE_MODEL}")
    print(f"  \U0001f4cb Tools: {allowed_tools or 'all'}")
    print(f"  " + "-" * 58)

    proc = subprocess.Popen(
        cmd, cwd=cwd,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True, bufsize=1,
    )
    proc.stdin.write(prompt)
    proc.stdin.close()

    while True:
        if proc.poll() is not None:
            # Drain remaining output
            for line in (proc.stdout.read() or '').split('\n'):
                _display_claude_line(line)
            for line in (proc.stderr.read() or '').split('\n'):
                if line.strip():
                    print(f"  \u2699\ufe0f  {line}", flush=True)
            break

        try:
            readable, _, _ = select.select([proc.stdout, proc.stderr], [], [], 0.1)
        except (ValueError, OSError):
            break

        for stream in readable:
            line = stream.readline()
            if line:
                if stream == proc.stdout:
                    _display_claude_line(line.rstrip('\n'))
                else:
                    if line.strip():
                        print(f"  \u2699\ufe0f  {line.rstrip()}", flush=True)

    rc = proc.wait()
    print(f"  " + "-" * 58)
    if rc != 0:
        print(f"  \u26a0\ufe0f  Claude exited with code {rc}")
    return rc

## Setup and verify the environment

In [3]:
import shutil

# ---------- Paths ----------
# Auto-detect REPO_DIR: walk up from notebook location to find project root
_nb_dir = os.path.abspath("")
if os.path.basename(_nb_dir) == "notebooks":
    REPO_DIR = os.path.dirname(_nb_dir)
else:
    REPO_DIR = _nb_dir

# ---------- Load API key from .env if not set ----------
if not ANTHROPIC_API_KEY:
    _env_file = os.path.join(REPO_DIR, ".env")
    if os.path.exists(_env_file):
        with open(_env_file) as f:
            for line in f:
                line = line.strip()
                if line.startswith("ANTHROPIC_API_KEY="):
                    ANTHROPIC_API_KEY = line.split("=", 1)[1].strip().strip("\"'")
                    break
        if ANTHROPIC_API_KEY:
            print(f"Loaded ANTHROPIC_API_KEY from {_env_file}")

if ANTHROPIC_API_KEY:
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
    print(f"Using API key (ends with ...{ANTHROPIC_API_KEY[-4:]})")
else:
    print("No API key set — Claude CLI will use your logged-in account.")

# ---------- Data paths ----------
EXAMPLE_DIR = os.path.join(REPO_DIR, "examples", "case2_binder_design")
DATA_DIR    = os.path.join(REPO_DIR, "data", TARGET_NAME)
RESULTS_DIR = os.path.join(REPO_DIR, "results", f"{TARGET_NAME}_binder")
TARGET_PDB  = os.path.join(DATA_DIR, f"{TARGET_NAME}.pdb")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "designs"), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "logs"), exist_ok=True)

# Copy example PDB into DATA_DIR
src_pdb = os.path.join(EXAMPLE_DIR, f"{TARGET_NAME}.pdb")
if os.path.exists(src_pdb) and not os.path.exists(TARGET_PDB):
    shutil.copy2(src_pdb, TARGET_PDB)
    print(f"Copied {src_pdb} \u2192 {TARGET_PDB}")

assert os.path.exists(TARGET_PDB), f"Target PDB not found: {TARGET_PDB}"

print(f"\nCLAUDE_MODEL   : {CLAUDE_MODEL}")
print(f"TARGET_NAME    : {TARGET_NAME}")
print(f"TARGET_CHAINS  : {TARGET_CHAINS}")
print(f"BINDER_LENGTH  : {BINDER_LENGTH}")
print(f"NUM_DESIGNS    : {NUM_DESIGNS}")
print(f"REPO_DIR       : {REPO_DIR}")
print(f"DATA_DIR       : {DATA_DIR}")
print(f"RESULTS_DIR    : {RESULTS_DIR}")
print(f"TARGET_PDB     : {TARGET_PDB}")

No API key set — Claude CLI will use your logged-in account.

CLAUDE_MODEL   : claude-sonnet-4-6
TARGET_NAME    : PDL1
TARGET_CHAINS  : A
BINDER_LENGTH  : 130
NUM_DESIGNS    : 3
REPO_DIR       : /home/xux/Desktop/AgentMCP/ProteinMCP
DATA_DIR       : /home/xux/Desktop/AgentMCP/ProteinMCP/data/PDL1
RESULTS_DIR    : /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder
TARGET_PDB     : /home/xux/Desktop/AgentMCP/ProteinMCP/data/PDL1/PDL1.pdb


## Install & Register MCPs

> **Tip — Local Docker build:** The `bindcraft_mcp` Docker image is large and may time out
> during `pmcp install`. You can build and register it locally instead:
> ```bash
> cd tool-mcps/bindcraft_mcp && docker build -t bindcraft_mcp:latest . && cd ../..
> claude mcp add bindcraft_mcp -- docker run -i --rm --gpus all --ipc=host -v $PWD:$PWD bindcraft_mcp:latest
> ```
> The install cell below will skip if `bindcraft_mcp` is already registered.

In [4]:
import time
_t0 = time.time()

# Install ProteinMCP if not already present
if run_cmd("which pmcp") != 0:
    run_cmd(f"pip install -e {REPO_DIR}")
    run_cmd(f"pip install -r {REPO_DIR}/requirements.txt")
else:
    print("ProteinMCP already installed.")

# Install Claude Code if not already present
if run_cmd("which claude") != 0:
    run_cmd("npm install -g @anthropic-ai/claude-code")
else:
    print("Claude Code already installed.")

# Verify Docker is available (required for bindcraft_mcp)
if run_cmd("docker --version") != 0:
    print("WARNING: Docker not found. bindcraft_mcp requires Docker with GPU support.")
else:
    print("Docker found.")

print(f"\nProteinMCP & Claude Code ready.")
print(f"Elapsed: {time.time() - _t0:.1f}s")

/home/xux/miniforge3/envs/protein-mcp/bin/pmcp
ProteinMCP already installed.
/home/xux/.local/bin/claude
Claude Code already installed.
Docker version 29.2.1, build a5c7197
Docker found.

ProteinMCP & Claude Code ready.
Elapsed: 0.0s


In [5]:
import time
_t0 = time.time()

# Check if bindcraft_mcp is already registered
_registered = subprocess.run(["claude", "mcp", "list"], capture_output=True, text=True).stdout
if "bindcraft_mcp" not in _registered:
    print("Installing bindcraft_mcp (this may take a while for the Docker image pull)...")
    run_cmd("pmcp install bindcraft_mcp", cwd=REPO_DIR)
else:
    print("bindcraft_mcp already registered. Skipping installation.")

# Verify MCP status
print(f"\n{'='*50}")
run_cmd("claude mcp list", cwd=REPO_DIR)
print(f"\nElapsed: {time.time() - _t0:.1f}s")

bindcraft_mcp already registered. Skipping installation.

Checking MCP server health...

msa_mcp: python /mnt/data/done_projects/2026/BioMolMCP/ProteinMCP/tool-mcps/msa_mcp/src/server.py - ✓ Connected
bindcraft_mcp: docker run -i --rm --user 129052:1129052 --gpus all --ipc=host -v /home/xux/Desktop/AgentMCP/ProteinMCP:/home/xux/Desktop/AgentMCP/ProteinMCP bindcraft_mcp:latest - ✓ Connected
plmc_mcp: docker run -i --rm --user 129052:1129052 --gpus all --ipc=host -v /home/xux/Desktop/AgentMCP/ProteinMCP:/home/xux/Desktop/AgentMCP/ProteinMCP plmc_mcp:latest - ✓ Connected
ev_onehot_mcp: docker run -i --rm --user 129052:1129052 --gpus all --ipc=host -v /home/xux/Desktop/AgentMCP/ProteinMCP:/home/xux/Desktop/AgentMCP/ProteinMCP ev_onehot_mcp:latest - ✓ Connected
prottrans_mcp: docker run -i --rm --user 129052:1129052 --gpus all --ipc=host -e HOME=/home/xux -v /home/xux/Desktop/AgentMCP/ProteinMCP:/home/xux/Desktop/AgentMCP/ProteinMCP -v /home/xux/.cache:/home/xux/.cache prottrans_mcp:latest 

## Step 1 — Explore Default Configurations

In [6]:
import time
_t0 = time.time()

prompt = f"""\
Can you show me the available default configurations for BindCraft using the \
bindcraft_mcp server? I want to understand what settings are available for \
binder design.
"""

run_claude(
    prompt,
    allowed_tools="mcp__bindcraft_mcp__get_default_configs,Bash,Read",
    cwd=REPO_DIR,
)

print(f"\nElapsed: {time.time() - _t0:.1f}s")

  🤖 Claude model: claude-sonnet-4-6
  📋 Tools: mcp__bindcraft_mcp__get_default_configs,Bash,Read
  ----------------------------------------------------------
  🤖 Session started: 43604eff...
  💭 Thinking: The user wants to see the available default configurations for BindCraft. Let me look at the bindcra...
  🔧 Glob
  🔧 Glob
  ✅ Result: No files found
  ✅ Result: No files found
  🔧 Glob
  ✅ Result: No files found
  🔧 Bash: ls tool-mcps/ 2>/dev/null || echo "No tool-mcps directory"
  ✅ Result: alphafold2_mcp
  🔧 Bash: find tool-mcps/bindcraft_mcp -type f | sort
  ✅ Done
  🔧 Bash: ls -la tool-mcps/bindcraft_mcp/
  ✅ Result: total 84
  🔧 Bash: ls tool-mcps/bindcraft_mcp/configs/
  🔧 Bash: ls tool-mcps/bindcraft_mcp/src/
  ✅ Result: default_config.json
  ✅ Result: bindcraft_mcp.py
  📖 Read: /home/xux/Desktop/AgentMCP/ProteinMCP/tool-mcps/bindcraft_mcp/configs/default_config.json
  ✅ Result: 1→{
  🔧 Bash: ls tool-mcps/bindcraft_mcp/src/tools/
  ✅ Result: bindcraft_config.py
  📖 Read: /home/

## Step 2 — Generate Configuration

In [7]:
import time
_t0 = time.time()

CONFIG_DIR = os.path.join(RESULTS_DIR, "config")

prompt = f"""\
Generate a BindCraft configuration for my target protein at {TARGET_PDB}. \
Target chain(s) {TARGET_CHAINS} and aim for a binder of length {BINDER_LENGTH}. \
Save the config to the output directory {CONFIG_DIR}. \
IMPORTANT: The output_file parameter is an output DIRECTORY path, not a file path. \
Please convert relative paths to absolute paths before calling the MCP server.
"""

run_claude(
    prompt,
    allowed_tools="mcp__bindcraft_mcp__generate_config,Bash,Read,Write",
    cwd=REPO_DIR,
)

# Verify config was created — generate_config creates output_dir/target_settings.json
config_settings = os.path.join(CONFIG_DIR, "target_settings.json")

# Find target_settings.json: check expected location, then search recursively
config = None
if os.path.isfile(config_settings):
    with open(config_settings) as f:
        config = json.load(f)
    print(f"\nConfig generated: {config_settings}")
elif os.path.isdir(CONFIG_DIR):
    # Search for target_settings.json recursively under CONFIG_DIR
    for root, dirs, files in os.walk(CONFIG_DIR):
        if "target_settings.json" in files:
            found = os.path.join(root, "target_settings.json")
            with open(found) as f:
                config = json.load(f)
            print(f"\nConfig generated: {found}")
            break
elif os.path.isfile(CONFIG_DIR):
    with open(CONFIG_DIR) as f:
        config = json.load(f)
    print(f"\nConfig generated: {CONFIG_DIR}")

if config:
    print(f"Config keys: {list(config.keys())}")
else:
    print(f"\nWARNING: Config not found at {CONFIG_DIR}")
    print("The design job in the next step will use default settings.")

print(f"Elapsed: {time.time() - _t0:.1f}s")

  🤖 Claude model: claude-sonnet-4-6
  📋 Tools: mcp__bindcraft_mcp__generate_config,Bash,Read,Write
  ----------------------------------------------------------
  🤖 Session started: 5bdcc599...
  💭 Thinking: The user wants to generate a BindCraft configuration for a target protein. Let me call the generate_...
  🔌 MCP: mcp__bindcraft_mcp__generate_config
  ✅ Result: {"status":"success","success":true,"result":{"files_generated":["target_settings
  Configuration generated successfully. Here's a summary:
  **Output directory:** `/home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder/config/`
  **Files generated:**
  ... (16 more lines)
  ✅ Completed successfully
  ----------------------------------------------------------

Config generated: /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder/config/target_settings.json
Config keys: ['design_path', 'binder_name', 'starting_pdb', 'chains', 'target_hotspot_residues', 'lengths', 'number_of_final_designs', '_metadata']
Elapsed: 29.4s


## Step 3 — Submit Binder Design Job

In [13]:
import time, re
_t0 = time.time()

# Find the settings file from generate_config output
_settings_file = os.path.join(RESULTS_DIR, "config", "target_settings.json")
if os.path.isfile(_settings_file):
    config_arg = f"Use the config file at {_settings_file}."
else:
    config_arg = ""

prompt = f"""\
Can you submit an async binder design job for {TARGET_PDB} using BindCraft? \
Generate {NUM_DESIGNS} designs targeting chain(s) {TARGET_CHAINS} with binder \
length {BINDER_LENGTH}. Save results to {RESULTS_DIR}/designs/. {config_arg}
Please convert relative paths to absolute paths before calling the MCP server.
After submitting, print the job_id so I can monitor progress.
"""

# Capture Claude output to extract job_id
proc = subprocess.Popen(
    ["claude", "--model", CLAUDE_MODEL, "-p", "-",
     "--output-format", "stream-json", "--verbose",
     "--dangerously-skip-permissions",
     "--allowedTools", "mcp__bindcraft_mcp__submit_async_design,Bash,Read,Write"],
    cwd=REPO_DIR,
    stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
    text=True, bufsize=1,
)
proc.stdin.write(prompt)
proc.stdin.close()

full_output = ""
for line in proc.stdout:
    full_output += line
    _display_claude_line(line.rstrip('\n'))
proc.wait()

# Try to extract job_id from output
job_id = None
for match in re.finditer(r'job_id["\']?\s*[:=]\s*["\']?([a-zA-Z0-9_-]+)', full_output):
    job_id = match.group(1)
if not job_id:
    # Try UUID pattern
    for match in re.finditer(r'[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}', full_output):
        job_id = match.group(0)

if job_id:
    print(f"\nCaptured job_id: {job_id}")
else:
    print("\nWARNING: Could not extract job_id from output.")
    print("You can manually set it: job_id = 'your-job-id-here'")

print(f"Elapsed: {time.time() - _t0:.1f}s")

  🤖 Session started: 68ade38d...
  💭 Thinking: The user wants to submit an async binder design job using BindCraft. Let me first check if the confi...
  🔧 Bash: ls /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder/config/ 2>/dev/null
  ✅ Result: default_4stage_multimer.json
  Config file exists. Submitting the async job now.
  🔌 MCP: mcp__bindcraft_mcp__bindcraft_submit
  ✅ Result: {"status":"submitted","message":"BindCraft job submitted successfully. Use bindc
  Job submitted successfully. Here's the summary:
  | Field | Value |
  |---|---|
  | **Status** | `submitted` |
  ... (15 more lines)
  ✅ Completed successfully

Captured job_id: cafbc064-5400-4725-b02f-fe5d9614dd35
Elapsed: 40.7s


## Step 4 — Monitor Job Progress

In [12]:
import time
_t0 = time.time()

if not job_id:
    print("No job_id set. Please set it manually: job_id = 'your-job-id'")
else:
    prompt = f"""\
Can you check the status of my BindCraft design job with ID {job_id}? \
Also show me the recent log output. \
Then list all BindCraft jobs and their statuses.
"""

    run_claude(
        prompt,
        allowed_tools="mcp__bindcraft_mcp__get_job_status,mcp__bindcraft_mcp__get_job_log,mcp__bindcraft_mcp__list_jobs,Bash,Read",
        cwd=REPO_DIR,
    )

print(f"\nElapsed: {time.time() - _t0:.1f}s")

  🤖 Claude model: claude-sonnet-4-6
  📋 Tools: mcp__bindcraft_mcp__get_job_status,mcp__bindcraft_mcp__get_job_log,mcp__bindcraft_mcp__list_jobs,Bash,Read
  ----------------------------------------------------------
  🤖 Session started: 7dcd069c...
  💭 Thinking: The user wants to:
1. Check the status of a specific BindCraft job with ID bdaea823-eaee-4f1e-bdf2-3...
  I'll search for the BindCraft job directory and list all BindCraft jobs in parallel.
  🔧 Bash: find /home/xux /tmp /var/tmp -name "*.log" -path "*bindcraft*" -o -name "*.log" 
  🔧 Bash: find /home/xux/Desktop/AgentMCP/ProteinMCP -type d 2>/dev/null | head -30; ls /t
  ✅ Result: /home/xux/Desktop/AgentMCP/ProteinMCP
  ✅ Done
  🔧 Bash: cat /tmp/bindcraft_docker_analysis.md 2>/dev/null | head -50; cat /tmp/restart_b
  ✅ Result: # BindCraft Docker Permission Issue Analysis
  🔧 Bash: find /home/xux -name "bindcraft_jobs*" -o -name "*bindcraft*jobs*" 2>/dev/null |
  ✅ Done
  🔧 Bash: find /home/xux/Desktop/AgentMCP/ProteinMCP/resul

## Step 5 — Retrieve Results

In [10]:
import time
_t0 = time.time()

if not job_id:
    print("No job_id set. Please set it manually: job_id = 'your-job-id'")
else:
    prompt = f"""\
Can you get the results of my completed BindCraft job with ID {job_id}? \
List all output files in {RESULTS_DIR}/designs/ and print any available \
design quality metrics (pLDDT, pAE, interface scores).
"""

    run_claude(
        prompt,
        allowed_tools="mcp__bindcraft_mcp__get_job_result,Bash,Read",
        cwd=REPO_DIR,
    )

# List output files
designs_dir = os.path.join(RESULTS_DIR, "designs")
if os.path.isdir(designs_dir):
    files = os.listdir(designs_dir)
    print(f"\nDesign outputs ({len(files)} files): {files}")

# Print metrics if available
for name in ["final_design_stats.csv", "mpnn_design_stats.csv", "design_metrics.csv", "metrics.csv"]:
    for search_dir in [designs_dir, RESULTS_DIR]:
        csv_path = os.path.join(search_dir, name)
        if os.path.exists(csv_path):
            import pandas as pd
            df = pd.read_csv(csv_path)
            print(f"\nMetrics from {name}:")
            print(df.to_string(index=False))
            break

print(f"\nElapsed: {time.time() - _t0:.1f}s")

  🤖 Claude model: claude-sonnet-4-6
  📋 Tools: mcp__bindcraft_mcp__get_job_result,Bash,Read
  ----------------------------------------------------------
  🤖 Session started: cab79bcc...
  💭 Thinking: The user wants to check the results of a completed BindCraft job. Let me check the status of the job...
  🔌 MCP: mcp__bindcraft_mcp__bindcraft_check_status
  ✅ Result: {"status":"success","job_status":"not_started","output_dir":"/home/xux/Desktop/A
  🔧 Bash: ls -lah /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder/designs/ 2>/de
  ✅ Result: total 44K
  🔧 Bash: ls -lah /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder/designs/Accept
  ✅ Result: total 24K
  🔧 Bash: cat /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder/designs/final_desi
  ✅ Result: Rank,Design,Protocol,Length,Seed,Helicity,Target_Hotspot,Sequence,InterfaceResid
  🔧 Bash: cat /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder/designs/bindcraft_
  ✅ Result: mkdir -p failed for path /.config/matp

## Step 6 — Visualize Results

In [11]:
import time
_t0 = time.time()

VIZ_SCRIPT = os.path.join(REPO_DIR, "workflow-skills", "scripts", "binder_design_viz.py")

# Install viz deps if needed
run_cmd("pip install -q matplotlib seaborn scipy Pillow pandas")

# Generate separate figures + merged summary
run_cmd(f"python {VIZ_SCRIPT} {RESULTS_DIR}")
run_cmd(f"python {VIZ_SCRIPT} {RESULTS_DIR} --merged")

# Display figures inline
from IPython.display import display, Image

# Try merged summary first
summary_png = os.path.join(RESULTS_DIR, "binder_design_summary.png")
if os.path.exists(summary_png):
    print("\nMerged summary figure:")
    display(Image(filename=summary_png, width=900))

# Also display individual figures
fig_names = [
    "binder_design_plddt_comparison.png",
    "binder_design_interface_pae.png",
    "binder_design_quality_scatter.png",
    "binder_design_design_ranking.png",
    "binder_design_metrics_table.png",
    "binder_design_execution_timeline.png",
]
for fname in fig_names:
    fpath = os.path.join(RESULTS_DIR, fname)
    if os.path.exists(fpath):
        print(f"\n{fname}:")
        display(Image(filename=fpath, width=500))

print(f"\nElapsed: {time.time() - _t0:.1f}s")

Loaded BindCraft metrics from /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder/designs/final_design_stats.csv (0 designs)
Error: No design metrics found in /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder

Visualization failed

⚠️  Command exited with code 1
Loaded BindCraft metrics from /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder/designs/final_design_stats.csv (0 designs)
Error: No design metrics found in /home/xux/Desktop/AgentMCP/ProteinMCP/results/PDL1_binder

Visualization failed

⚠️  Command exited with code 1

Elapsed: 3.6s


---
## Instructions & Troubleshooting

### Workflow Overview

This notebook uses BindCraft to design protein binders through an async job pipeline:

1. **Explore Configs** — View available BindCraft configuration templates
2. **Generate Config** — Create an optimized config for your target
3. **Submit Job** — Launch an async GPU design job (RFdiffusion + ProteinMPNN + AlphaFold2)
4. **Monitor Progress** — Poll job status and view logs
5. **Retrieve Results** — Download designed binder structures and metrics
6. **Visualize** — Generate quality assessment figures
7. **Download** — Package results for sharing

### Input Format

- **Target PDB** — Clean PDB file with proper chain IDs, waters/heteroatoms removed
- **Target chains** — Chain ID(s) to design binders against (e.g., `"A"` or `"A,B"`)
- **Binder length** — Desired length of the designed binder protein (typical: 60–200)

### Common Issues

| Problem | Solution |
|---------|----------|
| `bindcraft_mcp` not found | Run `pmcp install bindcraft_mcp` to pull Docker image |
| Job stuck in pending | Check GPU availability and Docker GPU runtime |
| Low quality designs (low pLDDT) | Try different binder lengths or specify hotspot residues |
| GPU out of memory | Reduce binder length or use smaller model config |
| Config generation fails | Verify chain IDs exist in PDB; try default settings |
| Job ID not captured | Manually set `job_id = 'your-id'` from Step 3 output |
| Docker image pull timeout | Re-run `pmcp install bindcraft_mcp` (large image ~15GB) |

### Quality Thresholds

| Metric | Good | Acceptable | Description |
|--------|------|------------|-------------|
| pLDDT | \u226580 | \u226570 | Predicted structure confidence (higher is better) |
| pAE | \u22644\u00c5 | \u226410\u00c5 | Predicted aligned error (lower is better) |
| i_pAE | \u226410 | \u226415 | Interface pAE (lower is better) |
| i_pTM | \u22650.6 | \u22650.4 | Interface pTM score (higher is better) |

### References

- [BindCraft](https://github.com/martinpacesa/BindCraft) — De novo binder design
- [RFdiffusion](https://github.com/RosettaCommons/RFdiffusion) — Structure diffusion
- [ProteinMPNN](https://github.com/dauparas/ProteinMPNN) — Inverse folding
- [AlphaFold](https://github.com/google-deepmind/alphafold) — Structure prediction
- [ProteinMCP](https://github.com/charlesxu90/ProteinMCP) — This project